# Cut-in detection

In this notebook, the cut ins are detected. 

In [ ]:
import cutin_detection

In [ ]:
%debug

In [ ]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp, kendalltau, spearmanr
from domain_model import StateVariable
from databaseemulator import DataBaseEmulator
from stats import KDE

In [ ]:
# Load the database with the cut-in scenarios.
filename = os.path.join("data", "5_cutin_scenarios", "database.json")
DBE = DataBaseEmulator(filename)
print("Number of scenarios: {:d}".format(len(DBE.collections["scenario"])))

In [ ]:
def cutin_parameters(scenario):
    # Obtain the duration of the lane change.
    activity = next(activity for activity in scenario.activities 
                    if activity.activity_category.name == 'lane change target')
    duration = activity.tduration
    ystart = activity.get_state(time=scenario.time["start"])[0]
    yend = activity.get_state(time=scenario.time["end"])[0]
    tstart = activity.tstart
    from_direction = 'r' if activity.name == "left lane change" else 'r'
    
    # Obtain longitudinal speed/position at time of start cut in.
    vstart, vend = None, None
    xstart = None
    init_activity_target = ''
    for activity in scenario.activities:
        if activity.activity_category.state == StateVariable.LON_TARGET and \
                activity.tstart <= scenario.time["start"] <= activity.tend:
            vstart = activity.get_state(time=scenario.time["start"])[0][0]
            vend = activity.get_state(time=activity.tend)[0][0]
            xstart = activity.get_state(time=scenario.time["start"])[1][0]
            if activity.activity_category.name == "deceleration target":
                init_activity_target = 'd'
            elif activity.activity_category.name == "acceleration target":
                init_activity_target = 'a'
            else:
                init_activity_target = 'c'
            tstartlon = activity.tstart
            tendlin = activity.tend
            break
            
    # Obtain ego vehicle speed at time of start cut in.
    vego = None
    for actor, activity, _ in scenario.acts:
        if actor.name == "ego vehicle" and \
            activity.activity_category.state == StateVariable.SPEED and \
                activity.tstart <= scenario.time["start"] <= activity.tend:
            vego = activity.get_state(time=scenario.time["start"])[0]
            
    return [duration, ystart, yend, xstart, vstart, vego,
            from_direction, init_activity_target, tstartlon, tendlin]

In [ ]:
# Get parameters of the cut-in: [duration, ystart, yend, xstart]
nscenarios = len(DBE.collections["scenario"])
#pars = [cutin_parameters(DBE.get_item("scenario", i)) for i in range(nscenarios)]
df = pd.DataFrame([cutin_parameters(DBE.get_item("scenario", i)) for i in range(nscenarios)], 
                  columns=['duration', 'ystart', 'yend', 'xstart', 'vstart', 'vego',
                           'from_direction', 'init_activity_target', 'tstartlon', 'tendlin'])
df["vdiff"] = df["vstart"] - df["vego"]
df["ystartabs"] = np.abs(df["ystart"])
df["yspeed"] = (df["yend"] - df["ystart"]) / df["duration"]
df["yspeedabs"] = np.abs(df["yspeed"])

# Plot a cut-in scenario

In [ ]:
def plot_cutin(scenario, axes=None):
    if axes is None:
        _, axes = plt.subplots(1, 1)
        axes.plot(0, 0, 'gx')
    df = pd.DataFrame(index=np.arange(scenario.time["start"], scenario.time["end"], 0.01),
                      columns=["x", "y", "act"])
    activity = next(activity for activity in scenario.activities 
                    if activity.activity_category.name == 'lane change target')
    df["y"] = activity.get_state(time=df.index.values)
    
    for activity in scenario.activities:
        if activity.activity_category.state == StateVariable.LON_TARGET:
            if activity.tstart < df.index[0]:
                i = 0
            else:
                i = df.index.get_loc(activity.tstart, method='ffill')
            if activity.tend > df.index[-1]:
                j = len(df)
            else:
                j = df.index.get_loc(activity.tend, method='bfill')
            df.loc[df.index[i]:df.index[j-1], "x"] = \
                activity.get_state(time=np.array(df.index[i:j]))[1]
            if activity.activity_category.name == "acceleration target":
                df.loc[df.index[i]:df.index[j-1], "act"] = "a"
            elif activity.activity_category.name == "deceleration target":
                df.loc[df.index[i]:df.index[j-1], "act"] = "d"
            elif activity.activity_category.name == "cruising target":
                df.loc[df.index[i]:df.index[j-1], "act"] = "c"
            else:
                raise ValueError("Unknown longitudinal activity")
    for act, color in zip(["a", "d", "c"], ["g", "r", "b"]):
        mask = df["act"] == act
        if np.sum(mask):
            axes.plot(df.loc[mask, "x"], df.loc[mask, "y"], color=color)
    xmax = axes.get_xlim()[1]
    axes.set_xlim([-10, xmax])
    return axes

In [ ]:
axes = plot_cutin(DBE.get_item("scenario", 0))
for i in range(1, 20):
    plot_cutin(DBE.get_item("scenario", i), axes=axes)

# Step 1: Initial tags

In [ ]:
for activity, abbr in zip(["acceleration", "deceleration", "cruising"], ["a", "d", "c"]):
    print("Probability of {:12s}: {:4.1f} %".
          format(activity, np.sum(df["init_activity_target"] == abbr)/nscenarios*100))

Furthermore, the initial lateral activity is always a lane change.

# Step 2: Initial parameters

The initial parameters consists of:

- `ystart`: Initial lateral position w.r.t. center lane of ego
- `xstart`: Initial longitudinal offset w.r.t. ego vehicle
- `tlongitudinal`: Time at which the longitudinal activity starts

In [ ]:
# Determine if the parameters depend on the initial longitudinal activity.
def get_ks_result(act1, act2, signal):
    values1 = df.loc[df["init_activity_target"] == act1, signal].values
    values2 = df.loc[df["init_activity_target"] == act2, signal].values
    result = ks_2samp(values1, values2)
    return result.pvalue

print("  Variable  KS test")
print("              a-d    a-c    d-c")
for signal in ['ystart', 'yend', 'tstartlon', 'ystartabs']:
    print("{:>10s}".format(signal), end="")
    for act1, act2 in zip(["a", "a", "d"], ["d", "c", "c"]):
        print("  {:5.3f}".format(get_ks_result(act1, act2, signal)), end="")
    print("")

In [ ]:
def test_independent(data1, data2):
    data = np.concatenate((data1, data2))
    k, k1, k2 = KDE(data=data), KDE(data=data1), KDE(data=data2)
    k.compute_bandwidth(), k1.compute_bandwidth(), k2.compute_bandwidth()
    print("1 KDE:  {:7.2f}".format(k.score_leave_one_out(include_const=True)))
    print("2 KDes: {:7.2f}".format(k1.score_leave_one_out(include_const=True) + 
                                   k2.score_leave_one_out(include_const=True)))
data1 = df.loc[df["init_activity_target"] == "d", "yend"].values
data2 = df.loc[df["init_activity_target"] == "c", "yend"].values
test_independent(data1, data2)

In [ ]:
np.random.seed(0)
data1 = np.random.randn(20) - 3
data2 = np.random.randn(20) + 3
data = np.concatenate((data1, data2))
k, k1, k2 = KDE(data=data), KDE(data=data1), KDE(data=data2)
k.compute_bandwidth(), k1.compute_bandwidth(), k2.compute_bandwidth()

In [ ]:
xpdf = np.linspace(-7, 7, 100)
ypdf, ypdf1, ypdf2 = k.score_samples(xpdf), k1.score_samples(xpdf), k2.score_samples(xpdf)
plt.plot(xpdf, ypdf)
plt.plot(xpdf, ypdf1)
plt.plot(xpdf, ypdf2)

In [ ]:
np.sum(np.log(k.score_samples(data)))

In [ ]:
np.sum(np.log(k1.score_samples(data1)))

In [ ]:
np.sum(np.log(k2.score_samples(data2)))

In [ ]:
k2.score_leave_one_out(include_const=True)

In [ ]:
def compute_score(data, combinations):
    score = 0
    for combination in combinations:
        kde = KDE(data=data[:, combination])
        kde.compute_bandwidth()
        score += kde.score_leave_one_out(include_const=True)
    return score
print("a=ystart, b=xstart, c=tstartlon")
for act in ["a", "d", "c"]:
    print("Initial activity: {:s}".format(act))
    data = df.loc[df["init_activity_target"] == act, 
                  ["ystart", "xstart", "tstartlon"]].values
    data /= np.std(data, axis=0)
    scores = [compute_score(data, combinations) 
              for combinations in [([0, 1, 2],), ([0, 1], 2), ([0, 2], 1), (0, [1, 2]), (0, 1, 2)]]
    i_best = np.argmax(scores)
    for i, (text, score) in \
            enumerate(zip(['(a,b,c)', '(a,b),(c)', '(a,c),(b)', '(a),(b,c)', '(a),(b),(c)'], scores)):
        print("{:>12s}:  {:.3e}{:>10s}".format(text, score, "Best!" if i==i_best else ""))

Conclusion: The parameters `ystart`, `xstart` and `tstartlon` will be sampled independently from each other. Furthermore, a different KDE is used for each initial longitudinal activity.

# Parameters activities

For the lane change, we have the parameters `yend` and `duration`. Alternatively, `yspeed` might be used instead of one of these.

For the longitudinal activity, we have the parameters `tend`, `vstart`, and `vend`. 

In [ ]:
# Determine if the parameters depend on the initial longitudinal activity.
def get_ks_result(act1, act2, signal):
    values1 = df.loc[df["init_activity_target"] == act1, signal].values
    values2 = df.loc[df["init_activity_target"] == act2, signal].values
    result = ks_2samp(values1, values2)
    return result.pvalue

print("  Variable  KS test")
print("              a-d    a-c    d-c")
for signal in ['duration', 'yend', 'yspeed', 'yspeedabs']:
    print("{:>10s}".format(signal), end="")
    for act1, act2 in zip(["a", "a", "d"], ["d", "c", "c"]):
        print("  {:5.3f}".format(get_ks_result(act1, act2, signal)), end="")
    print("")

In [ ]:
df['ystart']

In [ ]:
def get_ks_result(act1, act2, signal):
    values1 = df.loc[df["init_activity_target"] == act1, signal].values
    values2 = df.loc[df["init_activity_target"] == act2, signal].values
    result = ks_2samp(values1, values2)
    return result.pvalue

print("  Variable  KS test")
print("              a-d    a-c    d-c")
for signal in ['duration', 'ystart', 'yend', 'ystartabs']:
    print("{:>10s}".format(signal), end="")
    for act1, act2 in zip(["a", "a", "d"], ["d", "c", "c"]):
        print("  {:5.3f}".format(get_ks_result(act1, act2, signal)), end="")
    print("")

In [ ]:
for act in ['a', 'd', 'c']:
    n = np.sum(df["init_activity_target"] == act)
    plt.plot(np.sort(df.loc[df["init_activity_target"] == act, "ystart"]),
             np.linspace(0, 1, n), label=act)
plt.legend()

In [ ]:
# See if the lane change data is correlated.
print(spearmanr(df["ystart"], df["duration"]))
print(kendalltau(df["ystart"], df["duration"]))
print(spearmanr(df["yend"], df["duration"]))
print(kendalltau(df["yend"], df["duration"]))
print(spearmanr(df["yend"], df["ystart"]))
print(kendalltau(df["yend"], df["ystart"]))

# Step 4: Next activity

In [ ]:
# Compute 'transition' matrix
A = np.zeros((3, 3))  # 'a', 'd', 'c'
for i in range(nscenarios):
    s = DBE.get_item("scenario", i)
    t_acts = []
    lon_acts = []
    for actor, activity, tstart in s.acts:
        if actor.actor_category.name == "cut-in vehicle":
            if activity.activity_category.state == StateVariable.LON_TARGET:
                t_acts.append(tstart)
                lon_acts.append(activity)
    if len(t_acts) <= 1:
        continue
    i_sorted = np.argsort(t_acts)
    for i, j in zip(i_sorted[:-1], i_sorted[1:]):
        ia = (0 if lon_acts[i].activity_category.name == "acceleration target" else
              1 if lon_acts[i].activity_category.name == "deceleration target" else 2)
        ja = (0 if lon_acts[j].activity_category.name == "acceleration target" else
              1 if lon_acts[j].activity_category.name == "deceleration target" else 2)
        A[ia, ja] += 1

In [ ]:
A